#### 1. Import relevant model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display

#### 2. Create model

for simple model and visualization our samples are in one dimension $X=(x_1,x_2,...,x_n)^T \in \mathbb{R}^n$, our labels are in one dimension $Y=(y_1,y_2,...,y_n)^T \in \mathbb{R}^n$ and we have $d$ parameters $\theta=(x_1,x_2,...,x_d)^T \in \mathbb{R}^d$

* **general_model(X, w)** - generate polynomial model , for parameters $\theta \in \mathbb{R}^d$ we create model as follows :
\begin{equation}
h(\theta \; ; x) = \sum_{m=0}^{d-1} x^m \theta_{m}  \quad , x \in X
\end{equation} 
for $d=2$ we get linear model, 

* **loss(y, y_pred)** - Mean quere error loss.
\begin{equation}
\mathcal{L} (\theta) = \frac{1}{n} \sum_{(x,y) \in D} \left(h(\theta \; ; x) - y \right)^2 = \frac{1}{n} \sum_{(x,y) \in D} \left(\sum_{m=0}^{d-1} \theta_{m} x^m - y \right)^2   \quad , x \in X 
\end{equation}
> where $D$ is tuple of feature and his correspond label $D = \{ (x_i , y_i) \}_{k=1}^n$

* **grad(X, y, y_pred, w)** - the anlytical loss for of the loss function.

\begin{equation}
\frac{\partial \mathcal{L} (\theta)}{ \partial \theta_m} = \frac{2 x^m}{n} \sum_{(x,y) \in D} \left(h(\theta \; ; x) - y \right)  \quad ,\nabla_d \mathcal{L} (\theta) = \left( \frac{\partial \mathcal{L} (\theta)}{ \partial \theta_1},\frac{\partial \mathcal{L} (\theta)}{ \partial \theta_2},...,\frac{\partial \mathcal{L} (\theta)}{ \partial \theta_d}\right)^T
\end{equation} 

In [2]:
def general_model(X, w):
    loss = np.zeros_like(X)
    for i, val in enumerate(w):
        loss = loss + val * (X ** i)
    return loss

def loss(y, y_pred):
    return ((y_pred - y) ** 2).mean()

def grad(X, y, y_pred, w):
    grad = []
    for i, val in enumerate(w):
        grad.append((2 * (X ** i) * (y_pred - y)).mean())
    return np.array(grad)

#### 3. Generate data 

the features are sample from $x \sim U[a,b]  \quad , \text{X_range} = [a,b]$

* **generate_general_data(X_range, n, real_w, noise_power=0.2)** - generate data according to the real polynomial model with "noised" parameters $\theta$ at power of noise_power
\begin{equation}
y(x_m) = x^m (\theta_{m}^\star + Z_m) \quad, m=1,2,...,n  \quad, Z_m \sim U[-p,p]  \quad ,p = \text{noise_power}
\end{equation}

* **generate_binary_data(X_range, n)** - bineary labels $y \in \{0,1}\$

$$
y(x_m)=
\begin{cases}
    v_0, & \text{if } x_m < \frac{a+b}{2} \\
    v_1, & \text{otherwise}
\end{cases}
$$

** generate_sigmoid(X_range, n, r) ** - create sigmoid labels.

\begin{equation}
y(x_m) = \frac{1}{1+r^{x_m}} + Z_m \quad ,Z_m \sim U[-p,p]  \quad ,p = \text{noise_val}
\end{equation}

* **generate_window(X_range, n, r=10e-2, p_1=0.2, p_2=0.8, noise_val=0.1)** create window labels.

\begin{equation}
y(x_m) = \frac{1}{1+r^{(x_m - A)}} + \frac{1}{1+r^{(x_m - B)}} + Z_m \quad ,A = a + (b-a)p_1  , \quad B = a+(b-a)p_2
\end{equation}

In [3]:
def generate_general_data(X_range, n=200, real_w=[0, 1], noise_power=0.2):
        X = X_range[0] + (X_range[1] - X_range[0]) * np.random.rand(n)
        y = np.zeros(shape=(X.shape[0],))

        for i, w in enumerate(real_w):
            noised_w = w + np.random.uniform(low=-noise_power, high=noise_power, size=n)
            y = y + noised_w * (X ** i)
        return X, y

def generate_binary_data(X_range, n, v_0=0, v_1=1, noise_val=0.1):
        X = X_range[0] + (X_range[1] - X_range[0]) * np.random.rand(n)
        y = np.where(X < (X_range[1] + X_range[0]) / 2, v_0, v_1)
        return X, y + np.random.uniform(low=-noise_val, high=noise_val, size=n)

def generate_sigmoid(X_range, n, r=10e-3, noise_val=0.1):
        X = X_range[0] + (X_range[1] - X_range[0]) * np.random.rand(n)
        y = 1 / (1 + r ** X)
        return X, y + np.random.uniform(low=-noise_val, high=noise_val, size=n)

def generate_window(X_range, n, r=10e-5, p_1=0.2, p_2=0.8, noise_val=0.1):
        X = X_range[0] + (X_range[1] - X_range[0]) * np.random.rand(n)
        A = X_range[0] + (X_range[1] - X_range[0]) * p_1
        B = X_range[0] + (X_range[1] - X_range[0]) * p_2
        y = 1 / (1 + r **  (X - A)) - 1 / (1 + r ** (X - B))
        return X, y + np.random.uniform(low=-noise_val, high=noise_val, size=n)

#### 4. Generate our model and plotting result

In [8]:
class Model:
        def __init__(self, X, y, num_of_params=2, epochs=30, lr=0.2, batch_size=None):
            self.X = X
            self.y = y
            self.epochs = epochs
            self.batch_size = len(y) if batch_size is None else batch_size
            self.lr = lr
            self.w = np.random.uniform(low=-3, high=3, size=num_of_params)

            self.prev_params = []
            self.losses = []

            self.X_min = min(X) - 0.1 * (max(X) - min(X))
            self.X_max = max(X) + 0.1 * (max(X) - min(X))
            self.y_min = min(y) - 0.1 * (max(y) - min(y))
            self.y_max = max(y) + 0.1 * (max(y) - min(y))

            self.text = 'GD' if self.batch_size >= len(y) else f'SGD (batch = {self.batch_size})'
            self.fig, self.ax = plt.subplots(figsize=(16, 10))

        def plot_data(self, fig_size=(10, 6)):
            plt.plot(self.X, self.y, 'o')
            plt.xlabel('x' ,fontsize=14 ,fontweight="bold")
            plt.ylabel('y',fontsize=14 ,fontweight="bold")
            plt.title('Data',fontsize=14 ,fontweight="bold")
            plt.grid()

        def take_batch(self):
            idx = np.random.choice(list(range(len(self.y))), size=self.batch_size, replace=False)
            X_batch = np.array([self.X[i] for i in idx])
            y_batch = np.array([self.y[i] for i in idx])
            return X_batch, y_batch

        def guesses_update(self, frame):
            self.ax.clear()
            self.ax.set_xlim(self.X_min, self.X_max)
            self.ax.set_ylim(self.y_min, self.y_max)
            data = []
            line, = self.ax.plot(self.X, self.y, 'ko', ms=6)
            data.append(line)

            if self.batch_size < len(self.y):
                X, y = self.take_batch()
                line, = self.ax.plot(X, y, 'ro', ms=10, fillstyle='none')
                data.append(line)
            
            center = np.array([(self.X_min + self.X_max) / 2, (self.y_min + self.y_max) / 2])

            b = center[1] - frame * center[0]

            line, = self.ax.plot([self.X_min, self.X_max], [frame * self.X_min + b,  frame * self.X_max + b], 'b', ms=3)

            if self.batch_size < len(self.y):
               self.ax.set_title(f'bias : {b:.2f},  slope : {frame:.2f} \n' \
                                 f'full data loss  : {loss(self.y, frame * self.X + b):.2f}\n' 
                                 f'batch data loss : {loss(y, frame * X + b):.4f}', fontsize=14)
            else:
               self.ax.set_title(f'bias : {b:.2f},  slope : {frame:.2f} \n' \
                                 f'full data loss  : {loss(self.y, frame * self.X + b):.4f}', fontsize=14)
            self.ax.grid()
            return data

        def fit_update(self, frame):
            self.ax.clear()
            self.ax.set_xlabel('x' ,fontsize=16 ,fontweight="bold")
            self.ax.set_ylabel('y' ,fontsize=16 ,fontweight="bold")
            self.ax.set_xlim(self.X_min, self.X_max)
            self.ax.set_ylim(self.y_min, self.y_max)
            self.ax.grid()

            data = []
            line, = self.ax.plot(self.X, self.y, 'ko', ms=6)
            data.append(line)
            if self.batch_size < len(self.y):
                X, y = self.take_batch()
                line, = self.ax.plot(X, y, 'ro', ms=10, fillstyle='none')
                data.append(line)
            else:
                X, y = self.X, self.y

            y_pred = general_model(X, self.w)
            curr_loss = loss(y, y_pred)

            self.prev_params.append(list(np.round(self.w, 4)))
            self.losses.append(curr_loss)

            self.w = self.w - self.lr * grad(X, y, y_pred, self.w)

            vector = np.linspace(start=self.X_min, stop=self.X_max, num=50)
            line, = self.ax.plot(vector, general_model(vector, self.w), 'b', lw=3)
            round_w = list(np.round(self.w, 2))

            self.ax.set_title(f'{self.text}, epoch : {frame} \n' \
                              f'parms : {round_w} \n' \
                              f'loss  : {curr_loss:.4f}', fontsize=14)
            return data
        
        def make_linear_guess_video(self):
            res = 50
            anim = FuncAnimation(self.fig, self.guesses_update, frames=np.linspace(start=-20, stop=20, num=res), interval=100, blit=True)
            video = anim.to_html5_video()
            html = display.HTML(video)
            display.display(html)
            plt.close()

        def make_fit_video(self):
            anim = FuncAnimation(self.fig, self.fit_update, frames=list(range(self.epochs + 1)), interval=150, blit=True)
            video = anim.to_html5_video()
            html = display.HTML(video)
            display.display(html)
            plt.close()
            return self.w

        def print_log(self):
            for i, data in enumerate(zip(self.prev_params, self.losses)):
                print(f'epoch : {i} \n' \
                      f'params : {data[0][:10]} \n' \
                      f'loss : {data[1]:.4f} \n')

        def plot_loss(self, fig_size=(10, 6)):
            if len(self.losses):
                plt.figure(figsize=fig_size)
                plt.plot(np.linspace(start=0, stop=len(self.losses), num=len(self.losses)), self.losses, lw=3)
                plt.xlabel('epoch' ,fontsize=14 ,fontweight="bold")
                plt.ylabel('Loss',fontsize=14 ,fontweight="bold")
                plt.title('Loss',fontsize=14 ,fontweight="bold")
                plt.grid()


In [ ]:
all_data = [generate_general_data(X_range=[-1, 1], n=200, real_w=[0, 1], noise_power=0.2),
            generate_general_data(X_range=[-1, 1], n=200, real_w=[1, 2, -1, 0.5], noise_power=0.2),
            generate_binary_data(X_range=[-1, 1], n=200, v_0=0, v_1=1, noise_val=0.1),
            generate_sigmoid(X_range=[-1, 1], n=200, r=10e-5, noise_val=0.1),
            generate_window(X_range=[-1, 1], n=200, r=10e-5, p_1=0.2, p_2=0.8, noise_val=0.1)]

# Model.make_linear_guess_video() # optional plot linear buess video
# Model.print_log()               # optional print losses
# Model.plot_loss()               # optional plot loss

In [12]:
model_ = Model(*all_data[1], num_of_params=4, epochs=50, lr=0.2, batch_size=50).make_fit_video()
# model_.make_linear_guess_video() # optional plot linear buess video
# model_.print_log()               # optional print losses
# model_.plot_loss()               # optional plot loss